In [3]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import ast
from datetime import datetime
import seaborn as sns

def load_twitter_data(url):
    """
    Load Twitter data with proper parsing of list fields
    """
    try:
        df = pd.read_csv(url)
        # Convert string representations of lists to actual lists
        df['hashtags'] = df['hashtags'].apply(ast.literal_eval)
        df['mentions'] = df['mentions'].apply(ast.literal_eval)
        df['expanded_urls'] = df['expanded_urls'].apply(ast.literal_eval)
        print(f"Loaded {len(df)} tweets")
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

def create_interaction_network(df):
    """
    Create a network based on mentions, retweets, and replies
    """
    G = nx.DiGraph()
    
    for _, row in df.iterrows():
        source_user = row['user_key']
        
        # Add source user node
        if not G.has_node(source_user):
            G.add_node(source_user)
        
        # Add edges for mentions
        for mentioned_user in row['mentions']:
            if not G.has_node(mentioned_user):
                G.add_node(mentioned_user)
            G.add_edge(source_user, mentioned_user, type='mention')
        
        # Add edges for replies
        if pd.notna(row['in_reply_to_status_id']) and row['in_reply_to_status_id'] != "":
            reply_to = df[df['tweet_id'] == row['in_reply_to_status_id']]['user_key'].values
            if len(reply_to) > 0:
                if not G.has_node(reply_to[0]):
                    G.add_node(reply_to[0])
                G.add_edge(source_user, reply_to[0], type='reply')
        
        # Add edges for retweets
        if pd.notna(row['retweeted_status_id']) and row['retweeted_status_id'] != "":
            retweeted = df[df['tweet_id'] == row['retweeted_status_id']]['user_key'].values
            if len(retweeted) > 0:
                if not G.has_node(retweeted[0]):
                    G.add_node(retweeted[0])
                G.add_edge(source_user, retweeted[0], type='retweet')
    
    return G

def analyze_network(G):
    """
    Perform network analysis and return key metrics
    """
    metrics = {
        'num_nodes': G.number_of_nodes(),
        'num_edges': G.number_of_edges(),
        'density': nx.density(G),
        'avg_degree': sum(dict(G.degree()).values()) / G.number_of_nodes(),
    }
    
    # Calculate various centrality metrics
    metrics['degree_cent'] = nx.degree_centrality(G)
    metrics['in_degree_cent'] = nx.in_degree_centrality(G)
    metrics['out_degree_cent'] = nx.out_degree_centrality(G)
    
    # Find top users by different metrics
    metrics['top_influential'] = sorted(metrics['degree_cent'].items(), 
                                      key=lambda x: x[1], reverse=True)[:10]
    metrics['top_mentioned'] = sorted(metrics['in_degree_cent'].items(), 
                                    key=lambda x: x[1], reverse=True)[:10]
    metrics['top_active'] = sorted(metrics['out_degree_cent'].items(), 
                                 key=lambda x: x[1], reverse=True)[:10]
    
    return metrics

def visualize_network(G, metrics, title="Twitter Interaction Network"):
    """
    Create and save network visualization with different edge colors for different types
    """
    plt.figure(figsize=(15, 10))
    
    # Use spring layout for visualization
    pos = nx.spring_layout(G, k=1.5, iterations=50)
    
    # Draw nodes
    node_sizes = [3000 * metrics['degree_cent'][node] for node in G.nodes()]
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', 
                          node_size=node_sizes, alpha=0.7)
    
    # Draw edges with different colors based on type
    edge_colors = {'mention': 'blue', 'reply': 'green', 'retweet': 'red'}
    
    for edge_type, color in edge_colors.items():
        edges = [(u, v) for (u, v, d) in G.edges(data=True) 
                if d.get('type') == edge_type]
        nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color=color, 
                             alpha=0.4, arrows=True, arrowsize=10)
    
    # Add legend
    plt.plot([], [], 'b-', label='Mention')
    plt.plot([], [], 'g-', label='Reply')
    plt.plot([], [], 'r-', label='Retweet')
    plt.legend()
    
    plt.title(title)
    plt.axis('off')
    plt.savefig('twitter_network.png', dpi=300, bbox_inches='tight')
    plt.close()

def analyze_tweet_patterns(df):
    """
    Analyze patterns in the tweet dataset
    """
    patterns = {
        'total_tweets': len(df),
        'unique_users': df['user_key'].nunique(),
        'hashtag_stats': {},
        'temporal_patterns': {}
    }
    
    # Analyze hashtags
    all_hashtags = [tag for tags in df['hashtags'] for tag in tags]
    patterns['hashtag_stats'] = {
        'total_hashtags': len(all_hashtags),
        'unique_hashtags': len(set(all_hashtags)),
        'top_hashtags': pd.Series(all_hashtags).value_counts().head(10).to_dict()
    }
    
    # Temporal analysis
    df['created_at'] = pd.to_datetime(df['created_str'])
    patterns['temporal_patterns'] = {
        'tweets_by_hour': df.groupby(df['created_at'].dt.hour).size().to_dict(),
        'tweets_by_day': df.groupby(df['created_at'].dt.dayofweek).size().to_dict()
    }
    
    return patterns

def generate_report(df, metrics, patterns):
    """
    Generate comprehensive analysis report
    """
    report = f"""
Twitter Network Analysis Report
=============================

Network Statistics:
-----------------
• Total Users in Network: {metrics['num_nodes']:,}
• Total Interactions: {metrics['num_edges']:,}
• Network Density: {metrics['density']:.4f}
• Average Interactions per User: {metrics['avg_degree']:.2f}

Top Influential Users (by Total Interactions):
------------------------------------------
"""
    for user, score in metrics['top_influential']:
        report += f"• {user}: {score:.4f}\n"
    
    report += "\nMost Mentioned Users:\n"
    for user, score in metrics['top_mentioned']:
        report += f"• {user}: {score:.4f}\n"
    
    report += "\nMost Active Users:\n"
    for user, score in metrics['top_active']:
        report += f"• {user}: {score:.4f}\n"
    
    report += "\nHashtag Analysis:\n"
    report += f"• Total Hashtags Used: {patterns['hashtag_stats']['total_hashtags']:,}\n"
    report += f"• Unique Hashtags: {patterns['hashtag_stats']['unique_hashtags']:,}\n"
    report += "\nTop Hashtags:\n"
    for hashtag, count in patterns['hashtag_stats']['top_hashtags'].items():
        report += f"• #{hashtag}: {count:,} uses\n"
    
    return report

def main():
    # URL to the dataset
    url = "https://nodeassets.nbcnews.com/russian-twitter-trolls/tweets.csv"
    
    # Load data
    df = load_twitter_data(url)
    
    if df is not None:
        # Create and analyze network
        G = create_interaction_network(df)
        metrics = analyze_network(G)
        patterns = analyze_tweet_patterns(df)
        
        # Generate visualizations
        visualize_network(G, metrics)
        
        # Generate report
        report = generate_report(df, metrics, patterns)
        
        # Save report
        with open('twitter_analysis_report.txt', 'w') as f:
            f.write(report)
        
        print("Analysis complete. Check twitter_network.png for visualization and twitter_analysis_report.txt for the report.")

if __name__ == "__main__":
    main()

Loaded 203451 tweets


/var/folders/87/dhv__9sj0yv1kz0pfv2ds9nh0000gn/T/ipykernel_31992/407309783.py:118: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig('twitter_network.png', dpi=300, bbox_inches='tight')


Analysis complete. Check twitter_network.png for visualization and twitter_analysis_report.txt for the report.
